In [ ]:
# Delete all indexes to start fresh.
curl -X DELETE http://localhost:9200/ubi_queries,ubi_events,judgments,search_quality_eval_query_sets

In [ ]:
# Create the UBI indexes
curl -X PUT http://localhost:9200/ubi_queries/_mappings -H "Content-Type: application/json" -d'
{
    "properties": {
      "timestamp": { "type": "date", "format": "strict_date_time" },
      "query_id": { "type": "keyword", "ignore_above": 100 },
      "query": { "type": "text" },
      "query_response_id": { "type": "keyword", "ignore_above": 100 },
      "query_response_hit_ids": { "type": "keyword" },
      "user_query": { "type": "keyword", "ignore_above": 256 },
      "query_attributes": { "type": "flat_object" },
      "client_id": { "type": "keyword", "ignore_above": 100 },
      "application":  { "type":  "keyword", "ignore_above": 100 }
    }
}'

curl -X PUT http://localhost:9200/ubi_events/_mappings -H "Content-Type: application/json" -d'
{
  "properties": {
    "application": { "type": "keyword", "ignore_above": 256 },
    "action_name": { "type": "keyword", "ignore_above": 100 },
    "client_id": { "type": "keyword", "ignore_above": 100 },
    "query_id": { "type": "keyword", "ignore_above": 100 },
    "message": { "type": "keyword", "ignore_above": 1024 },
    "message_type": { "type": "keyword", "ignore_above": 100 },
    "timestamp": {
      "type": "date",
      "format":"strict_date_time",
      "ignore_malformed": true,
      "doc_values": true
    },
    "event_attributes": {
      "dynamic": true,
      "properties": {
        "position": {
          "properties": {
            "ordinal": { "type": "integer" },
            "x": { "type": "integer" },
            "y": { "type": "integer" },
            "page_depth": { "type": "integer" },
            "scroll_depth": { "type": "integer" },
            "trail": { "type": "text",
              "fields": { "keyword": { "type": "keyword", "ignore_above": 256 }
              }
            }
          }
        },
        "object": {
          "properties": {
            "internal_id": { "type": "keyword" },
            "object_id": { "type": "keyword", "ignore_above": 256 },
            "object_id_field": { "type": "keyword", "ignore_above": 100 },
            "name": { "type": "keyword", "ignore_above": 256 },
            "description": { "type": "text",
              "fields": { "keyword": { "type": "keyword", "ignore_above": 256 } }
            },
            "object_detail": { "type": "object" }
          }
        }
      }
    }
  }
}'

In [ ]:
# Index the ESCI data.
cd ./../data/esci && ./index.sh 1> /dev/null 2> /dev/null

In [ ]:
# See the query and events count.
curl -s http://localhost:9200/ubi_queries/_count
curl -s http://localhost:9200/ubi_events/_count

In [ ]:
# Create implicit judgments now.
curl -s -X POST "http://localhost:9200/_plugins/search_quality_eval/judgments?click_model=coec&max_rank=20"

# Schedule implicit judgments creation.
# curl -s -X POST "http://localhost:9200/_plugins/search_quality_eval/schedule?id=1&click_model=coec&max_rank=20&job_name=test2&interval=10" | jq
# curl -s "http://localhost:9200/search_quality_eval_scheduler/_search" | jq

In [ ]:
# See the judgments.
curl -s http://localhost:9200/judgments/_search | jq

In [ ]:
# Create a query set using PPTSS sampling from the judgments.
curl -s -X POST "http://localhost:9200/_plugins/search_quality_eval/queryset?name=test&description=fake&sampling=pptss&query_set_size=5000"

In [ ]:
# Look at the query sets.
curl -s http://localhost:9200/search_quality_eval_query_sets/_search | jq

In [ ]:
# Run a query set.
curl -s http://localhost:9200/search_quality_eval_run?id=${QUERY_SET_ID}